In [1]:
# SQL, DF are just API , interfaces
import findspark
findspark.init()

In [2]:
from pyspark.conf import SparkConf
config = SparkConf()
config.setMaster("spark://192.168.11.71:7077").setAppName("SparkDataFrameGlobalTempView")
#config.setMaster("local[2]").setAppName("SparkDataFrameHdfs")
config.set("spark.executor.memory", "4g")
config.set("spark.executor.cores", 4)
config.set("spark.cores.max", 4)
config.set("spark.driver.memory", "4g")


from pyspark.sql import SparkSession
spark = SparkSession.builder.config(conf=config).getOrCreate()

In [3]:
from pyspark.sql.types import StructType, LongType,StringType, IntegerType, DoubleType

movieSchema = StructType()\
         .add("movieId", IntegerType(), True)\
         .add("title", StringType(), True)\
         .add("genres", StringType(), True)\


ratingSchema = StructType()\
         .add("userId", IntegerType(), True)\
         .add("movieId", IntegerType(), True)\
         .add("rating", DoubleType(), True)\
         .add("timestamp", StringType(), True)\

In [4]:
movieDf = spark.read.format("csv")\
          .option("header", True)\
          .schema(movieSchema)\
          .load("hdfs://192.168.93.128:9000/ml-latest-small/movies.csv")

ratingDf = spark.read.format("csv")\
          .option("header", True)\
          .schema(ratingSchema)\
          .load("hdfs://192.168.93.128:9000/ml-latest-small/ratings.csv")

In [6]:
movieDf.show(2)
ratingDf.show(2)

+-------+----------------+--------------------+
|movieId|           title|              genres|
+-------+----------------+--------------------+
|      1|Toy Story (1995)|Adventure|Animati...|
|      2|  Jumanji (1995)|Adventure|Childre...|
+-------+----------------+--------------------+
only showing top 2 rows

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
+------+-------+------+---------+
only showing top 2 rows



In [8]:
spark.sql("CREATE TABLE student (id INT, name STRING, age INT) USING CSV")

DataFrame[]

In [9]:
spark.sql("show tables in default").show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
| default|  student|      false|
+--------+---------+-----------+



In [10]:
spark.sql("""CREATE DATABASE mysparkdb
LOCATION 'c:/spark/productdb'""")

DataFrame[]

In [11]:
spark.sql("""DESCRIBE DATABASE mysparkdb""")

DataFrame[database_description_item: string, database_description_value: string]

In [13]:
spark.sql("SET spark.sql.warehouse.dir;").show()

+--------------------+-----------+
|                 key|      value|
+--------------------+-----------+
|spark.sql.warehou...|<undefined>|
+--------------------+-----------+



In [14]:
spark.sql("SHOW DATABASES").show()

+------------+
|databaseName|
+------------+
|     default|
|   mysparkdb|
+------------+



In [17]:
spark.sql("""CREATE TABLE IF NOT EXISTS  mysparkdb.students(name STRING, gender STRING) ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde' 
    STORED AS TEXTFILE""")

AnalysisException: "Hive support is required to CREATE Hive TABLE (AS SELECT);;\n'CreateTable `mysparkdb`.`students`, org.apache.hadoop.hive.serde2.OpenCSVSerde, Ignore\n"

In [18]:
spark.sql("CREATE TABLE test(id STRING)")

AnalysisException: "Hive support is required to CREATE Hive TABLE (AS SELECT);;\n'CreateTable `test`, org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe, ErrorIfExists\n"